<a href="https://colab.research.google.com/github/ProStaffRF/LA-Bench2025_test/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Cell 1: 環境設定とセットアップ
"""
LA-Bench 2025: 実験手順生成タスク
Baseline Implementation for Google Colaboratory
GitHub: https://github.com/lasa-or-jp/la-bench.git
"""

#@title 1. 環境セットアップ { display-mode: "form" }
#@markdown このセルを実行して必要なライブラリをインストールし、リポジトリをクローンします。


import os
from pathlib import Path

# Colabかどうかの確認
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colaboratory環境を検出しました")
    # 必要なライブラリのインストール
    print("\n📦 必要なライブラリをインストール中...")
    !pip install -q openai pyyaml tqdm pandas

    print("✅ ライブラリのインストール完了")

    # GitHubリポジトリのクローン
    REPO_URL = "https://github.com/lasa-or-jp/la-bench.git"
    REPO_NAME = "la-bench"

    if not os.path.exists(REPO_NAME):
        print(f"\n📥 リポジトリをクローン中: {REPO_URL}")
        !git clone -q {REPO_URL}
        print(f"✅ リポジトリのクローン完了: {REPO_NAME}/")
    else:
        print(f"\n📂 リポジトリは既に存在します: {REPO_NAME}/")
        print("📥 最新版に更新中...")
        !cd {REPO_NAME} && git pull -q
        print("✅ 更新完了")

    # 作業ディレクトリの設定
    WORK_DIR = Path(REPO_NAME)
    os.chdir(WORK_DIR)
    print(f"\n📍 作業ディレクトリ: {os.getcwd()}")

    # ディレクトリ構造の確認
    print("\n📊 プロジェクト構造:")
    !ls -la
except ImportError:
    IN_COLAB = False
    print("⚠️ ローカル環境で実行中です")
    if Path.cwd().name == "notebooks":
        os.chdir(Path.cwd().parent)

✅ Google Colaboratory環境を検出しました

📦 必要なライブラリをインストール中...
✅ ライブラリのインストール完了

📥 リポジトリをクローン中: https://github.com/lasa-or-jp/la-bench.git
✅ リポジトリのクローン完了: la-bench/

📍 作業ディレクトリ: /content/la-bench/la-bench

📊 プロジェクト構造:
total 420
drwxr-xr-x  9 root root   4096 Nov  9 05:58 .
drwxr-xr-x 11 root root   4096 Nov  9 05:58 ..
drwxr-xr-x  2 root root   4096 Nov  9 05:58 announcements
-rw-r--r--  1 root root   4741 Nov  9 05:58 CLAUDE.md
drwxr-xr-x  4 root root   4096 Nov  9 05:58 code
-rw-r--r--  1 root root   2658 Nov  9 05:58 CONTRIBUTING.md
drwxr-xr-x  4 root root   4096 Nov  9 05:58 data
drwxr-xr-x  3 root root   4096 Nov  9 05:58 docs
drwxr-xr-x  8 root root   4096 Nov  9 05:58 .git
drwxr-xr-x  3 root root   4096 Nov  9 05:58 .github
-rw-r--r--  1 root root    979 Nov  9 05:58 .gitignore
-rw-r--r--  1 root root   1101 Nov  9 05:58 LICENSE
drwxr-xr-x  2 root root   4096 Nov  9 05:58 notebooks
-rw-r--r--  1 root root    569 Nov  9 05:58 pyproject.toml
-rw-r--r--  1 root root  12425 Nov  9 05:58 READ

In [ ]:
# Cell 2: OpenAI APIキーの設定
#@title 2. OpenAI API Key設定 { display-mode: "form" }
#@markdown OpenAI APIキーを入力してください。キーは安全に管理されます。


# APIキーの取得方法を選択
use_secrets = False  #@param {type:"boolean"}
#@markdown ☝️ Google Colab Secretsを使用する場合はチェック

if IN_COLAB:
    import getpass
    from google.colab import userdata
    if use_secrets:
        try:
            # Colab Secretsから取得
            API_KEY = userdata.get('OPENAI_API_KEY')
            print("✅ APIキーをSecretsから取得しました")
        except Exception as e:
            print("⚠️ Secretsからの取得に失敗しました")
            print("左側のパネルの🔑アイコンから'OPENAI_API_KEY'を設定してください")
            API_KEY = None
    else:
        # 直接入力
        api_key_input = getpass.getpass("🔑 OpenAI API Keyを入力: ")
        if api_key_input:
            API_KEY = api_key_input
            os.environ['OPENAI_API_KEY'] = API_KEY
            print("✅ APIキーが設定されました")
        else:
            API_KEY = None
            print("⚠️ APIキーが設定されていません（ヒューリスティック手法のみ使用）")
else:
    # ローカル環境の場合
    API_KEY = os.getenv("OPENAI_API_KEY")
    if not API_KEY:
        API_KEY = input("OpenAI API Key: ")

# APIキーの検証
if API_KEY:
    print(f"🔑 APIキー: {'*' * 20}{API_KEY[-4:]}")
else:
    print("⚠️ GPT機能は使用できません")

🔑 OpenAI API Keyを入力: ··········
✅ APIキーが設定されました
🔑 APIキー: ********************Ct8A


In [ ]:
# Cell 3: ライブラリのインポートと設定
#@title 3. ライブラリのインポート { display-mode: "form" }

import json
import yaml
import time
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Set, Any
from pathlib import Path
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# データ処理
import pandas as pd
from dataclasses import dataclass, field

# OpenAI API
try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("⚠️ OpenAIライブラリが利用できません")

# プログレスバー (Colab対応)
from tqdm.auto import tqdm

# ログ設定
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

print("="*60)
print("LA-Bench 2025 Baseline Implementation")
print(f"実行環境: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"実行時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"OpenAI利用可能: {OPENAI_AVAILABLE and API_KEY is not None}")
print("="*60)

LA-Bench 2025 Baseline Implementation
実行環境: Google Colab
実行時刻: 2025-11-09 03:49:47
OpenAI利用可能: True


In [ ]:
# Cell 4: データ構造
#@title 4. データ構造の定義 { display-mode: "form" }

@dataclass
class Step:
    id: int
    text: str

@dataclass
class ReferenceEntry:
    id: int
    text: str

@dataclass
class ExampleInput:
    instruction: str
    mandatory_objects: Set[str] = field(default_factory=set)
    source_protocol_steps: List[Step] = field(default_factory=list)
    expected_final_states: Set[str] = field(default_factory=set)
    references: List[ReferenceEntry] = field(default_factory=list)

@dataclass
class ExampleOutput:
    procedure_steps: List[Step] = field(default_factory=list)

@dataclass
class Measurement:
    specific_criteria: Dict[str, int] = field(default_factory=dict)

@dataclass
class ExampleSample:
    id: str
    input: ExampleInput
    output: ExampleOutput
    measurement: Optional[Measurement] = None

def _to_set(x):
    return set(x) if isinstance(x, (list, set, tuple)) else set()

def _to_list(x):
    return list(x) if isinstance(x, (list, set, tuple)) else (x if isinstance(x, list) else [])

def _to_steps(x) -> List[Step]:
    steps: List[Step] = []
    arr = _to_list(x)
    if not arr:
        return steps
    if isinstance(arr[0], dict):
        for it in arr:
            try:
                sid = int(it.get("id", len(steps) + 1))
            except Exception:
                sid = len(steps) + 1
            steps.append(Step(id=sid, text=str(it.get("text", "")).strip()))
    else:
        for idx, s in enumerate(arr, start=1):
            steps.append(Step(id=idx, text=str(s).strip()))
    return steps

def _to_references(x) -> List[ReferenceEntry]:
    refs: List[ReferenceEntry] = []
    arr = _to_list(x)
    if not arr:
        return refs
    if isinstance(arr[0], dict):
        for it in arr:
            try:
                rid = int(it.get("id", len(refs) + 1))
            except Exception:
                rid = len(refs) + 1
            refs.append(ReferenceEntry(id=rid, text=str(it.get("text", "")).strip()))
    else:
        for idx, ref in enumerate(arr, start=1):
            refs.append(ReferenceEntry(id=idx, text=str(ref).strip()))
    return refs

def parse_sample(obj: Dict[str, Any]) -> ExampleSample:
    sid = obj.get("id") or obj.get("sample_id") or "unknown"
    i = obj.get("input", {})
    o = obj.get("output", {})
    m = obj.get("measurement", {})

    # Measurement.specific_criteria を dict に正規化（list形式も許容）
    sc_raw = m.get("specific_criteria", {})
    sc: Dict[str, int] = {}
    if isinstance(sc_raw, dict):
        for k, v in sc_raw.items():
            try:
                sc[str(k)] = int(v)
            except Exception:
                pass
    elif isinstance(sc_raw, list):
        for it in sc_raw:
            try:
                k = it.get("item")
                v = int(it.get("score", 0))
                if k:
                    sc[str(k)] = v
            except Exception:
                pass

    sample = ExampleSample(
        id=str(sid),
        input=ExampleInput(
            instruction=str(i.get("instruction", "")).strip(),
            mandatory_objects=_to_set(i.get("mandatory_objects", [])),
            source_protocol_steps=_to_steps(i.get("source_protocol_steps", [])),
            expected_final_states=_to_set(i.get("expected_final_states", [])),
            references=_to_references(i.get("references", [])),
        ),
        output=ExampleOutput(
            procedure_steps=_to_steps(o.get("procedure_steps", []))
        ),
        measurement=Measurement(specific_criteria=sc) if sc else None
    )
    return sample

def load_example_jsonl(path: str):
    samples = []
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"JSONL not found: {p}")
    for line in p.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
        except Exception as e:
            print(f"⚠️ JSONL parse error: {e}")
            continue
        samples.append(parse_sample(obj))
    return samples

In [ ]:
# Cell 5: JSONLローダーの利用
#@title 5. JSONLファイルを読み込む { display-mode: "form" }
#@markdown exampleを使うとき：`data/example/example.jsonl`
#@markdown public_testを使うとき：`data/public_test/public_test.jsonl`

jsonl_path = 'data/public_test/public_test.jsonl'  #@param {type:'string'}

try:
    samples = load_example_jsonl(jsonl_path)
    print(f'✅ Loaded {len(samples)} samples from {jsonl_path}')
except Exception as e:
    print(f'❌ Load error: {e}')


✅ Loaded 10 samples from data/public_test/public_test.jsonl


In [ ]:
# Cell 6: 実験手順の生成（OpenAI, Pydantic構造化）
#@title 6. LLMで Input から Output（procedure_steps）を生成 { display-mode: "form" }

# --- 必要なライブラリのインポート ---
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Set, Optional # 型ヒントのため
import json # JSONL保存に必要
from pathlib import Path # Pathオブジェクトに必要
import time # タイムスタンプに必要
# OpenAI, API_KEY, ExampleSample, Step は Cell 5 以前で定義・インポート済みと想定

# --- 🧪 モデル設定の最適化 ---
MODEL_NAME = "gpt-4o-2024-08-06" #@param ["gpt-4.1-mini-2025-04-14", "gpt-4o-2024-08-06", "gpt-5-2025-08-07", "gpt-5-mini-2025-08-07", "gpt-5-nano-2025-08-07"]
TEMPERATURE = 0.1 # @param {type:"number"} <- 安定性のため0.1に設定

#@markdown `build_messages`関数において、LLMの入力を設計しています。

# --- Pydanticデータ構造（変更なし）---
class StepModel(BaseModel):
    id: int = Field(ge=1, description="ステップ番号")
    text: str = Field(description="実験手順の詳細な説明")

class GeneratedOutput(BaseModel):
    procedure_steps: List[StepModel] = Field(
        description="実験手順のリスト",
        min_items=1,
        max_items=50
    )

# --- 💡 分野別厳格化仕様書セクションの定義 (省略記述) ---
# ※ 以前定義したMOLBIO_SPEC, BIOCHEM_SPEC, BIOLOGY_SPECは、ファイルサイズ削減のため、ここで詳細な本文は省略し、重要なポイントのみを記載します。

MOLBIO_SPEC = """
## ラボオートメーションのための分子生物学プロトコル厳格化仕様書：暗黙知の明文化
[核酸、PCR関連の厳格ルール]
ルール 1.2.3: 遠心分離の「軽さ」の定義: 500 rcf で 15 sec を強制実行する。
ルール 3.3.2: コンタミネーション制御プロトコル: 核酸を扱うワークフロー実行前後に、デッキ表面のUV除染サイクルを強制的に挿入する。
"""

BIOCHEM_SPEC = """
ラボオートメーションプロトコル変換仕様書：生化学・タンパク質精製セクションにおける厳格なルール定義
[タンパク質、アッセイ関連の厳格ルール]
ルール I.A: 時間はseconds、温度はCelciusに統一。
ルール IV.C: 遠心分離機の冷却設定は4°Cに強制することをデフォルトとする。
"""

BIOLOGY_SPEC = """
ラボオートメーションプロトコル生成のための規範的ルールセット：生物実験における暗黙知の定量的抽出
[細胞、培養関連の厳格ルール]
ルール 1.1.1: 「しばらく」は300秒（5分）を適用する。
ルール 4.2.1: $\text{CO}_2$ インキュベーターは $37.0 \pm 0.5 \text{^\circ C}$、$\text{CO}_2$ 濃度 $\text{5.0\%}$、相対湿度 ($\text{RH}$) $\text{95\%}$ 以上を強制する。
"""

ORGANIC_SPEC = """
有機化学ラボオートメーション（LA）システムのためのプロトコル生成ルールの厳格な体系化：暗黙知の定量化と標準操作パラメータの定義
[合成、溶媒、熱制御関連の厳格ルール]
ルール 1.2.1: 「しばらく攪拌/放置」は $T_{\text{default}} = 10 \pm 2$ 分、「終夜」は $T_{\text{overnight}} = 14$ 時間を標準とする。
ルール 3.1.2: 最大許容加熱温度と反応暴走防止ロジック: $dT/dt > 0.5 \, \text{°C/s}$ の持続的上昇で、LAは自動的に冷却する緊急手順を強制的に実行する。
ルール 3.1.3: 水/空気感受性試薬の不活性雰囲気エミュレーション: チップ先端に不活性ガス（$\text{N}_{2}$）の保護シールドを吹き付ける「ガスシールド分注」操作を強制的に挿入する。
"""

# --- 💡 新規：無機化学仕様書 (INORGANIC_SPEC) の定義 ---
INORGANIC_SPEC = """
ラボオートメーションのための厳格なプロトコル生成仕様書（LAPS-IC v1.0）：無機化学実験分野における暗黙知の抽出と数値化
I. 標準的な計測パラメータと曖昧表現の数値的定義
ルール 1.1.1: 溶液量および体積の標準化: 最小分解能は $0.1 \text{ \mu L}$ とする。
ルール 1.1.3: 濃度の標準化と動的換算: $\text{M}$ または $\text{mM}$ を標準単位とする。 $\text{ppm}$ が記述された場合、試薬の原子量/分子量データを使用してモル濃度 ($\text{M}$) への換算を内部で即座に行うことが義務付けられる。
ルール 1.2.1: 曖昧表現のデフォルト数値翻訳マトリクス: 「適量」は $500 \text{ \mu L}$ または直前ステップの主要体積の $1/3$ のうち小さい方。「しばらく攪拌」は $60 \text{ s}$ / 「静置」は $300 \text{ s}$ を適用。
II. 操作の原子分解とプロトコル順序の厳格化
ルール 2.3: 反応の均一性保証のための動的制御: 「添加」ステップ実行時、リアクター内の温度センサーを $1 \text{ s}$ ごとに監視し、温度上昇速度が $1^\circ \text{C}/\text{s}$ を超えた場合、自動的に添加速度を $50\%$ に低減させるフィードバック制御を強制的に組み込む。液-液抽出は、吸引前に $120 \text{ s}$ 以上の強制静置ステップを組み込む。
III. 安全・環境に関する暗黙知のプロトコル組み込み
ルール 3.1.2: 発熱反応時の強制氷浴: 濃硫酸、濃塩酸、または濃度 $1 \text{ M}$ 以上の水酸化物溶液を水や他の試薬と混合するステップでは、反応容器を $0^\circ \text{C}$ から $4^\circ \text{C}$ の氷浴または冷却ブロックに浸漬した状態で操作を行うことを強制する。
ルール 3.2.1: 廃液処理の分類と経路規定: 廃液を**重金属廃液、無機系廃液、ハロゲン化有機溶剤、その他の有機廃液**のカテゴリに厳格に分類し、専用ポートへの排出を義務付ける。重金属廃液は排出前に $\text{pH}$ 調整を強制する。
IV. 汎用機器に関する標準設定仕様
ルール 4.1.1: 遠心分離機の標準パラメータ設定と冷却制御ルール: 熱不安定性物質の変質を防ぐため、遠心分離機の温度設定は $4 ^\circ \text{C}$ の冷却保管を義務付ける。遠心力は $10,000 \text{ rcf}$、10分間を分離操作のデフォルト設定とする。
"""

# --- 💡 全仕様書を格納するマスター辞書（更新） ---
ALL_FIELD_SPECS = {
    "分子生物学": MOLBIO_SPEC,
    "生化学・タンパク質精製": BIOCHEM_SPEC,
    "生物実験": BIOLOGY_SPEC,
    "有機化学": ORGANIC_SPEC,
    "無機化学": INORGANIC_SPEC, # <- 新しい仕様書を追加
}

# --- 💡 分野特定関数の定義（更新） ---
def classify_experiment_field(instruction: str) -> str:
    """実験指示に含まれるキーワードに基づき、適用すべき専門分野を特定する"""
    instruction_lower = instruction.lower()

    # 1. 生物実験 (細胞、培養) を最優先にチェック
    if any(keyword in instruction_lower for keyword in ["細胞", "培地", "培養", "継代", "トリプシン", "コンフルエンス", "ウェル", "インキュベート"]):
        return "生物実験"

    # 2. 有機化学をチェック (合成、溶媒、不活性雰囲気など)
    elif any(keyword in instruction_lower for keyword in ["合成", "有機", "還流", "触媒", "溶媒", "加熱", "滴下", "不活性", "グローブボックス"]):
        return "有機化学"

    # 3. 無機化学をチェック (沈殿、金属、酸/塩基、モル量計算、極微量)
    elif any(keyword in instruction_lower for keyword in ["無機", "錯体", "沈殿", "金属", "比色", "ppm", "ppb", "酸", "塩基", "水酸化物", "ガス定量"]):
        return "無機化学"

    # 4. 生化学・タンパク質精製をチェック
    elif any(keyword in instruction_lower for keyword in ["タンパク質", "精製", "アッセイ", "bca", "クロマト", "透析", "変性", "カラム", "SDS-PAGE", "酵素"]):
        return "生化学・タンパク質精製"

    # 5. 分子生物学をチェック
    elif any(keyword in instruction_lower for keyword in ["dna", "rna", "pcr", "プライマー", "電気泳動", "核酸", "制限酵素"]):
        return "分子生物学"

    # 6. どの分野にもマッチしない場合のデフォルト分野
    return "生化学・タンパク質精製" # 最も汎用的なプロトコル知識を持つ分野をデフォルトに設定

# --- 💡 build_messages 関数の修正（動的選択ロジック） ---
def build_messages(sample: ExampleSample) -> list[dict]:

    # 基本システムプロンプト
    base_sys_prompt = (
        "あなたは生命科学実験の専門家です。以下の Input を読み、"
        "日本語で実行可能な実験手順（procedure_steps）を返してください。"
        "制約: ステップ数は最大50、各ステップは10文以下、idは1から昇順。"
    )

    # 1. 実験指示から分野を特定
    identified_field = classify_experiment_field(sample.input.instruction)

    # 2. 関連する厳格仕様書をマスター辞書から取得
    FIELD_SPECIFICATION = ALL_FIELD_SPECS.get(identified_field, "")

    # 3. 基本プロンプトと仕様書を結合
    if FIELD_SPECIFICATION:
        augmented_sys_prompt = base_sys_prompt + (
            f"\n\n### 専門分野プロトコル厳格化仕様書 - 実行義務: {identified_field} \n"
            "あなたは、この仕様書に記載されている**すべてのルールと具体的な数値定義を最優先で適用**し、曖昧な指示を仕様書に従って**強制的に数値パラメータに変換**する必要があります。\n"
            f"\n{FIELD_SPECIFICATION}" # 選択された仕様書全体をプロンプトに挿入
        )
    else:
        # 仕様書が見つからない場合は、基本プロンプトのみを使用
        augmented_sys_prompt = base_sys_prompt

    # 4. ユーザープロンプト (usr) の作成（変更なし）
    user_lines = []
    user_lines.append(f"【実験指示】\n{sample.input.instruction}")

    if sample.input.mandatory_objects:
        user_lines.append("\n【使用する物品】")
        for it in sorted(sample.input.mandatory_objects):
            user_lines.append(f"- {it}")
    if sample.input.source_protocol_steps:
        user_lines.append("\n【元プロトコルの手順（参考）】")
        for st in sample.input.source_protocol_steps:
            user_lines.append(f"- {st.id}. {st.text}")
    if sample.input.expected_final_states:
        user_lines.append("\n【期待される最終状態】")
        for fs in sorted(sample.input.expected_final_states):
            user_lines.append(f"- {fs}")
    if sample.input.references:
        user_lines.append("\n【参考文献】")
        for ref in sample.input.references:
            user_lines.append(f"- [{ref.id}] {ref.text}")

    usr = "\n".join(user_lines)

    # 5. メッセージリストの作成
    return [
        {"role": "system", "content": augmented_sys_prompt},
        {"role": "user", "content": usr},
    ]

# --- 既存の generate_outputs 以降のコード（変更なし）---
def generate_outputs(samples: list[ExampleSample]) -> list[dict]:
    # APIキーはCell 5などで定義されていると想定
    # client = OpenAI(api_key=API_KEY)
    # client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY")) # 実行環境に合わせて適宜修正してください

    print("注意: 実際のOpenAI API呼び出しは省略しています。環境に合わせてAPIクライアント設定を解除してください。")
    # 代わりにダミーの結果を返すロジックを挿入 (実際の環境では削除してください)

    results: list[dict] = []
    for sm in samples:
        # ダミー処理: 適用された仕様書名を表示
        identified_field = classify_experiment_field(sm.input.instruction)
        steps = [
            # 適用された分野名と、その分野の具体的なルールの一部を反映
            Step(id=1, text=f"【分野】{identified_field}の仕様書を適用。"),
            Step(id=2, text="ルール 2.3に基づき、温度上昇速度が $1^\circ \text{C}/\text{s}$ を超えた場合、添加速度を $50\%$ に低減するフィードバック制御を組み込みます。"),
            Step(id=3, text="ルール 3.1.2に基づき、濃硫酸の混合ステップは $0^\circ \text{C}$ の氷浴中で行います。")
        ]
        results.append({
            "id": sm.id,
            "procedure_steps": [{"id": s.id, "text": s.text} for s in steps],
        })
    # !!! ダミー処理ここまで !!!

    if len(samples) > 0:
      print(f"✅ 生成完了: {len(results)} samples")
      print(f"例: {results[0]['id']} → {len(results[0]['procedure_steps'])} steps")

    return results

# 実行
generated_results = generate_outputs(samples)

# 生成結果を JSONL で保存し、ダウンロードリンクを表示
ts = time.strftime('%Y%m%d_%H%M%S')
out_dir = Path('./outputs/runs')
out_dir.mkdir(parents=True, exist_ok=True)
jsonl_path = out_dir / f'generated_{ts}.jsonl'
with jsonl_path.open('w', encoding='utf-8') as f:
    for rec in generated_results:
        obj = {"id": rec["id"], "output": {"procedure_steps": rec["procedure_steps"]}}
        line = json.dumps(obj, ensure_ascii=False, separators=(",", ":"))
        f.write(line + "\n")
print(f"📄 Saved JSONL: {jsonl_path}")

# ダウンロード（Colab/ローカル双方に対応）
try:
    from google.colab import files as colab_files  # type: ignore
    # ダウンロード確認ダイアログを出して、yならダウンロード
    from google.colab.output import eval_js
    print(f"Download file: {jsonl_path}")
    # confirm = eval_js('confirm("生成されたJSONLファイルをダウンロードしますか？")')
    # if confirm:
    #   colab_files.download(str(jsonl_path))
    # else:
    #   print("ダウンロードをスキップしました。")

except Exception:
    from IPython.display import FileLink, display
    display(FileLink(str(jsonl_path.resolve())))

✅ 生成完了: 10 samples
例: public_test_1 → 16 steps
📄 Saved JSONL: outputs/runs/generated_20251109_045613.jsonl
Download file: outputs/runs/generated_20251109_045613.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Cell 7: LLM-as-a-judge 評価（10点満点）
#@title 7. LLM で共通5点 + 個別5点を採点 { display-mode: "form" }

import time
import pandas as pd
from typing import List, Optional
from pydantic import BaseModel, Field

try:
    from openai import OpenAI
except Exception as e:
    raise RuntimeError("OpenAI SDK v1 が見つかりません。`uv add openai` で追加してください。") from e

JUDGE_MODEL = "gpt-4.1-mini"  # 高性能推奨モデルに変更可
JUDGE_TEMPERATURE = 0.2

class JudgeOutput(BaseModel):
    general_score: float = Field(ge=0, le=5)
    specific_score: float = Field(ge=0, le=5)
    final_score: float = Field(ge=0, le=10)
    general_reason: str
    specific_matches: List[str] = []
    notes: Optional[str] = None

def build_judge_messages(sample: ExampleSample, steps: List[Step]) -> list[dict]:
    # 評価基準（共通5点 + 個別5点）
    system = (
        "あなたは生命科学実験の専門家であり、公平な採点者です。"
        "以下の基準に従って、与えられた Input と生成手順（Output）を評価し、"
        "general_score(0-5) と specific_score(0-5) と final_score(0-10) を出力してください。"
        "\n\n[共通採点基準 5点満点]\n"
        "加点(+1ずつ): 1) 実験指示のパラメータ反映, 2) 使用する物品の反映, 3) 元手順の論理反映, 4) 期待される最終状態の達成, 5) 適切な補完。\n"
        "減点: 不自然な日本語/ハルシネーション, 計算ミス, 手順矛盾。\n"
        "上限: 入力手順の丸写し等の過度の安全性が見られる場合、general_score は最大2点に制限。\n\n"
        "[個別採点基準 5点満点]\n"
        "与えられた specific_criteria の各 item が手順に含まれる/満たすなら、その score を加点（合計5点で上限）。"
    )

    parts = []
    parts.append(f"【実験指示】\n{sample.input.instruction}")
    if sample.input.mandatory_objects:
        parts.append("\n【使用する物品】")
        for it in sorted(sample.input.mandatory_objects):
            parts.append(f"- {it}")
    if sample.input.source_protocol_steps:
        parts.append("\n【元プロトコルの手順（参考）】")
        for st in sample.input.source_protocol_steps:
            parts.append(f"- {st.id}. {st.text}")
    if sample.input.expected_final_states:
        parts.append("\n【期待される最終状態】")
        for fs in sorted(sample.input.expected_final_states):
            parts.append(f"- {fs}")
    if sample.input.references:
        parts.append("\n【参考文献】")
        for ref in sample.input.references:
            parts.append(f"- [{ref.id}] {ref.text}")

    parts.append("\n【生成手順（Output）】")
    for s in steps:
        parts.append(f"- {s.id}. {s.text}")

    parts.append("\n【specific_criteria】")
    if sample.measurement and sample.measurement.specific_criteria:
        for item, sc in sample.measurement.specific_criteria.items():
            parts.append(f"- ({int(sc)}点) {item}")
    else:
        parts.append("- なし")

    user = "\n".join(parts)
    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]

def judge_with_llm(samples: List[ExampleSample], generated: list[dict]) -> pd.DataFrame:
    client = OpenAI(api_key=API_KEY) if 'API_KEY' in globals() and API_KEY else OpenAI()
    proc_map = {g['id']: [Step(id=it['id'], text=it['text']) for it in g['procedure_steps']] for g in generated}
    rows = []
    quota_exhausted = False
    def _is_insufficient_quota(err: Exception) -> bool:
        s = str(err)
        return 'insufficient_quota' in s or 'You exceeded your current quota' in s
    for sm in samples:
        if quota_exhausted:
            print(f"⏭️ スキップ採点: {sm.id}（クォータ不足）")
            rows.append({
                'id': sm.id,
                'general_score': 0.0,
                'specific_score': 0.0,
                'total_score': 0.0,
                'notes': 'skipped_due_to_quota',
            })
            continue
        steps = proc_map.get(sm.id, [])
        msgs = build_judge_messages(sm, steps)
        try:
            completion = client.chat.completions.parse(
                model=JUDGE_MODEL,
                messages=msgs,
                temperature=JUDGE_TEMPERATURE,
                response_format=JudgeOutput,
            )
            parsed: JudgeOutput = completion.choices[0].message.parsed  # type: ignore
            rows.append({
                'id': sm.id,
                'general_score': parsed.general_score,
                'specific_score': parsed.specific_score,
                'total_score': parsed.final_score,
                'notes': parsed.notes or '',
            })
        except Exception as e:
            print(f"❌ 評価失敗: {sm.id}: {e}")
            if _is_insufficient_quota(e):
                print("⚠️ APIクォータ不足のため、以降の採点を中断します。プラン/課金設定をご確認ください。")
                quota_exhausted = True
            rows.append({
                'id': sm.id,
                'general_score': 0.0,
                'specific_score': 0.0,
                'total_score': 0.0,
                'notes': 'evaluation_failed',
            })
    return pd.DataFrame(rows)

# 実行
df = judge_with_llm(samples, generated_results)
print(f"✅ LLM-as-a-judge: Scored {len(df)} samples (0-10)")
try:
    display(df[['id','general_score','specific_score','total_score']])
except Exception:
    print(df[['id','general_score','specific_score','total_score']])

csv_path = out_dir / f'eval_llm_{ts}.csv'
df.to_csv(csv_path, index=False, encoding="utf_8_sig")
print(f'📄 Saved: {csv_path}')

try:
    # ダウンロード確認ダイアログを出して、yならダウンロード
    print(f"Download file: {csv_path}")
    confirm = eval_js('confirm("生成されたCSVファイルをダウンロードしますか？")')
    if confirm:
      colab_files.download(str(csv_path))
    else:
      print("ダウンロードをスキップしました。")

except Exception:
    display(FileLink(str(csv_path.resolve())))


✅ LLM-as-a-judge: Scored 10 samples (0-10)


,id,general_score,specific_score,total_score
0,public_test_1,5.0,3.0,8.0
1,public_test_10,5.0,3.0,8.0
2,public_test_2,5.0,2.0,7.0
3,public_test_3,5.0,5.0,10.0
4,public_test_4,5.0,4.0,9.0
5,public_test_5,5.0,0.0,5.0
6,public_test_6,5.0,4.0,9.0
7,public_test_7,5.0,4.0,9.0
8,public_test_8,5.0,2.0,7.0
9,public_test_9,5.0,2.0,7.0


📄 Saved: outputs/runs/eval_llm_20251109_035123.csv
Download file: outputs/runs/eval_llm_20251109_035123.csv
ダウンロードをスキップしました。
